# INSTITUTO DE EDUCAÇÃO SUPERIOR – IESB

# PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS

# Data Mining e Machine Learning II

# Prof.: Marcos

# Aluno: André dos Santos Feitosa

# Matrícula:1931133005


# Contexto

O departamento de crédito ao consumidor de um banco deseja automatizar o processo de
tomada de decisão para aprovação das linhas de crédito do patrimônio líquido. Para fazer
isso, eles seguirão as recomendações da Lei da Igualdade de Oportunidades de Crédito para
criar um modelo de pontuação de crédito derivado empiricamente e estatisticamente sólido.
O modelo será baseado em dados coletados de solicitantes recentes concedidos crédito através
do processo atual de subscrição de empréstimos. O modelo será construído a partir de ferramentas
de modelagem preditiva, mas o modelo criado deve ser suficientemente interpretável para fornecer
um motivo para qualquer ação adversa (rejeição).



# Conteúdo


O conjunto de dados de Home Equity (HMEQ) contém informações de linha de base e de desempenho de empréstimos
para 5.960 empréstimos recentes de home equity. O alvo (BAD) é uma variável binária que indica se um requerente
acabou por falhar ou se foi gravemente delinqüente. Esse desfecho adverso ocorreu em 1.189 casos (20%). Para cada
candidato, foram registradas 12 variáveis de entrada.




# Inspiração


E se você puder prever clientes que não pagam seus empréstimos?



In [ ]:
# Dados das Colunas



# MAU1 =  1 cliente inadimplente no empréstimo 0 = empréstimo reembolsado
# EMPRÉSTIMOMontante do pedido de empréstimo
# MORTDUEValor devido da hipoteca existente
# VALORValor da propriedade atual
# RAZÃODebtCon = consolidação da dívida HomeImp = melhoria da casa
# TRABALHOSeis categorias ocupacionais
# YOJAnos no emprego atual
# DEROGNúmero de principais relatórios depreciativos
# DELINQNúmero de linhas de crédito inadimplentes
# CLAGEIdade da linha comercial mais antiga em meses
# NINQNúmero de linhas de crédito recentes
# CLNONúmero de linhas de crédito
# DEBTINCRácio dívida / rendimento



In [ ]:
# Renomeação de dados #


# SITUACAO -  1 = cliente inadimplente no empréstimo 0 = empréstimo reembolsado
# VALOR_EMPRESTIMO - Montante do pedido de empréstimo
# VALOR_HIPT - Valor devido da hipoteca existente
# VALOR_PROP_ATUAL - valor da propriedade atual
# MOTIVO - consolidação da dívida HomeImp = melhoria da casa
# TRABALHO_PROF - Seis categorias profissionais (Manager, Office, Other, Prof.Executive, Sales, Self)
# TEMP_TRA - Anos no emprego atual
# N_REL_DREP - Número de principais relatórios depreciativos
# QTD_LINHAS_CREDITO_INAD - número de linhas de crédito inadimplentes
# IDADE_MAIS_ANTIGA_LN - Idade da linha comercial mais antiga em meses
# QTD_LINHAS_CREDITO_RECENT - Número de linhas de crédito recentes
# QTD_LINHAS_CREDITO - Número de linhas de crédito
# RECEITAS_DIVIDA_TX_MES - Taxa de receita da dívida (é a porcentagem da renda bruta mensal de um consumidor que é destinada ao pagamento de dívidas)



In [ ]:
# importação das bibliotecas

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Importação o arquivo e definição do nome

df = pd.read_csv("/kaggle/input/hmeq-data/hmeq.csv")

df.head(100)


# Tratamento da base #

In [ ]:
# Renomeando as colunas

df.columns = ['SITUACAO', 'VALOR_EMPRESTIMO', 'VALOR_HIPT', 'VALOR_PROP_ATUAL', 'MOTIVO', 'TRABALHO_PROF', 
              'TEMP_TRA', 'N_REL_DREP', 'QTD_LINHAS_CREDITO_INAD', 'IDADE_MAIS_ANTIGA_LN', 'QTD_LINHAS_CREDITO_RECENT', 
              'QTD_LINHAS_CREDITO', 'RECEITAS_DIVIDA_TX_MES']        


df.head()


In [ ]:
# imputação dos valores faltantes #

df.fillna(df.mean(), inplace=True)

In [ ]:
df.head(100)

# Análise Descritiva Exploratória (EDA) #

In [ ]:
# Quantitativo da variável SITUACAO

qtd = df['SITUACAO'].value_counts()

qtd

In [ ]:
#DEMONSTRAÇÃO GRÁFICA DA SITUACAO

%matplotlib inline
import seaborn as sns

df['SITUACAO'].value_counts().plot.bar()

In [ ]:
# Distribuição comparativa entre os domínios das variáveis SITUACAO, MOTIVO e TRABALHO_PROF


def showBalance(df, col):
    for c in col:
        print('Distribuição da Coluna: ', c,'\n',df[c].value_counts(normalize=True),'\n')
    else:
       pass
        
showBalance(df, col=['MOTIVO','TRABALHO_PROF','SITUACAO'])
  
      
 

In [ ]:
# Quantitativo por Tipo de Profissão 

df['TRABALHO_PROF'].value_counts().plot.bar()



Podemos observar que 68,81% dos que adquiriram o empréstimo foram para consolidação de dívida e cerca de 31,1% para melhoria na casa.

Os tipos de profissões mais representativas para o processo de crédito foram a Other com 42,03%, ProfExe com 22.46% e Office 16,68%.

Observando as situações dos empréstimos, é possível afirmar que cerca de 80,0% estão com as obrigações em dias, porém, cerca de 20% possui alguma condição de atraso.


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
fig = df.VALOR_EMPRESTIMO.hist(bins=30)
fig.set_title('Distribuição de Empréstimos (VALOR_EMPRESTIMO)')
fig.set_ylabel('Quantidade de Observações com os Empréstimos no eixo X')

Por meio do gráfico de distribuição do total de empréstiomos solicitados, verificamos
que o maior volume de solicitações são na base de 10.000 a 20.000.

In [ ]:
# Distribuição entre situacao e profissao

qtd1 = df.groupby(['SITUACAO'])['TRABALHO_PROF'].value_counts()

qtd1

In [ ]:
# Avaliando as variáveis categóricas em relacao ao pefil do pagador - GRÁFICO

TRABALHO_PROF=pd.crosstab(df['TRABALHO_PROF'],df['SITUACAO'])
TRABALHO_PROF.div(TRABALHO_PROF.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='Tipos de Empregos e Clientes', figsize=(8,8))


Observa-se por meio da análise gráfica que o grupo de vendas (Sales) e empreendedores (auto) possui número maior
de inadimplencia.



In [ ]:
MOTIVO=pd.crosstab(df['MOTIVO'],df['SITUACAO'])
MOTIVO.div(MOTIVO.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, title='Situações e Razões', figsize=(5,5))


Avaliando a variável MOTIVO x SITUAÇÃO, verifica-se que os valores estão próximos para as duas categorias.



# Tranformação de dados para aplicação nos modelos #

In [ ]:
# colunas que são do tipo object

df.select_dtypes('object').head()

In [ ]:
# Eliminando os NA #

df2 = df.copy()
df2.dropna(axis=0,how='any',inplace= True)
df2.info(), df2.isna().any() 
df2.shape



O tratamento dos campos com NA possibilitou a exclusão de 424 linhas. 



In [ ]:
# Tabela sem (NAN) 

df2.head(100)

In [ ]:
# Analisando a quantidade dos dados da coluna MOTIVO - Object

df2['MOTIVO'].value_counts()

In [ ]:
# Analisando a  quantiade dos dados da coluna TRABALHO_PROF - Object

df2['TRABALHO_PROF'].value_counts()


In [ ]:
# tranformando as colunas de object em categoria com codigos #

for col in df2.columns:
    if df2[col].dtype == 'object':
        df2[col]= df2[col].astype('category').cat.codes


In [ ]:
# Verificando a tranformação para dammy dos dados da coluna MOTIVO

df2['MOTIVO'].value_counts()


In [ ]:
# Verificando a tranformação para dammy  dos dados da coluna TRABALHO_PROF 

df2['TRABALHO_PROF'].value_counts()

# Geração Amostras de Treino e Teste #

In [ ]:
# importando a biblioteca

from sklearn.model_selection import train_test_split

In [ ]:
#Etapa 1- Primeiro Separando em Treino e Teste

treino, teste = train_test_split(df2, random_state=42)

treino.shape, teste.shape


In [ ]:
# Verificando os valores da variavel SITUACAO - TREINO

treino['SITUACAO'].value_counts(normalize=True)

In [ ]:
#Verificando os valores da variavel SITUACAO - TESTE

teste['SITUACAO'].value_counts(normalize=True)

# Aplicação das bases treino e teste #

In [ ]:
# separar as colunas para usar no modelo

usadas_treino = [c for c in treino.columns if c not in ['SITUACAO']]


# Modelo RandomForest 

In [ ]:
# Modelo RandomForest - Imputação / predição / acuracia  

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(treino[usadas_treino],treino['SITUACAO'])
rf_pred = rf.predict(teste[usadas_treino])
accuracy_score(teste['SITUACAO'], rf_pred)



In [ ]:
# verificação do resultado de predição

teste['SITUACAO'] = rf.predict(teste[usadas_treino])

teste['SITUACAO'].value_counts(normalize=True)

In [ ]:


# Após a aplicação das bases de treino e teste para predição do modelo RandomForest, pode-se observar que o mesmo obteve
# um nível de acuracia de 91%.

# O que pode ser demsotrado por meio dos valores da base de teste e predição da variavel SITUAÇÃO:
    
# BASE DE TESTE:
    
# 0    0.787572
# 1    0.212428


# PREDIÇÃO: 
    
# 0    0.833815
# 1    0.166185    




# Modelo GBM 

In [ ]:
# Modelo GBM - Imputação / predição / acuracia 

from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier(n_estimators=200, random_state=42)
gbm.fit(treino[usadas_treino], treino['SITUACAO'])
gbm_pred = gbm.predict(teste[usadas_treino])
accuracy_score(teste['SITUACAO'], gbm_pred)

In [ ]:
# verificação do resultado de predição

teste['SITUACAO'] = gbm.predict(teste[usadas_treino])

teste['SITUACAO'].value_counts(normalize=True)

In [ ]:


# Após a aplicação das bases de treino e teste para predição do modelo GBM, pode-se observar que o mesmo obteve
# um nível de acuracia de 96%.

# O que pode ser demostrado por meio dos valores da base de teste e predição da variavel SITUAÇÃO:
    
# BASE DE TESTE:
    
# 0    0.787572
# 1    0.212428


# PREDIÇÃO: 
    
# 0    0.825867
# 1    0.174133   




# Modelo XGBoost #

In [ ]:
# Modelo XGBoost - Imputação / predição / acuracia 
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=200, random_state=42)
xgb.fit(treino[usadas_treino], treino['SITUACAO'])
xbm_pred = xgb.predict(teste[usadas_treino])
accuracy_score(teste['SITUACAO'], xbm_pred)



In [ ]:
# verificação do resultado de predição

teste['SITUACAO'] = xgb.predict(teste[usadas_treino])

teste['SITUACAO'].value_counts(normalize=True)

In [ ]:


# Após a aplicação das bases de treino e teste para predição do modelo GBM, pode-se observar que o mesmo obteve
# um nível de acuracia de 94%.

# O que pode ser demostrado por meio dos valores da base de teste e predição da variavel SITUAÇÃO:
    
# BASE DE TESTE:
    
# 0    0.787572
# 1    0.212428


# PREDIÇÃO: 
    
# 0    0.817919
# 1    0.182081  




# Desempenho




O desempenho dos 3 modelos foram muito semelhantes, mesmo que o GBM tenha obtido o maior nível de acurácia em relação ao 
Random Forest e XGBoost.
No passo abaixo poderemos olhar o nível de importância que cada modelo definiu para as variáveis por meio dos gráficos.



# Avaliando a importância de cada variável nos modelos #


In [ ]:
# Verificando e avaliando a importancia de cada coluna para o modelo RF

pd.Series(rf.feature_importances_, index=usadas_treino).sort_values().plot.barh()



No modelo de RF as váriáveis mais consideradas para a realização da predição foram RECEITAS_DIVIDA_TX_MES,
IDADE_MAI_ANTIGA_LN e QTD_LINHAS_CREDITO_INAD.



In [ ]:
# Verificando e avaliando a importancia de cada coluna para o modelo GBM

pd.Series(gbm.feature_importances_, index=usadas_treino).sort_values().plot.barh()


No modelo de GBM as váriáveis mais importantes para a realização da predição foram: RECEITAS_DIVIDA_TX_MES,
QTD_LINHAS_CREDITO_INAD e IDADE_MAI_ANTIGA_LN.
 
Pode-se observar que houve uma inversão entre a 2º e a 3º.



In [ ]:
# Verificando e avaliando a importancia de cada coluna para o modelo XGB

pd.Series(xgb.feature_importances_, index=usadas_treino).sort_values().plot.barh()



No modelo de XGB as váriáveis mais importantes para a realização da predição foram: RECEITAS_DIVIDA_TX_MES,
QTD_LINHAS_CREDITO_INAD e N_REL_DREP.
 
A Variável IDADE_MAI_ANTIGA_LN foi alterada para a N_REL_DREP  como a 3º mais importante.





A situaçao das váriáveis analisadas demostra que todos os modelos foram dinamincos e chegram basicamente ao
mesmo resultado.



In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

# Matriz de Confusão - dados do modelo RandomForest 

skplt.metrics.plot_confusion_matrix(teste['SITUACAO'], rf_pred)

In [ ]:
# Matriz de Confusão - Dados do modelo GBM

skplt.metrics.plot_confusion_matrix(teste['SITUACAO'], gbm_pred)


In [ ]:
# Matriz de Confusão - Dados do modelo GBM

skplt.metrics.plot_confusion_matrix(teste['SITUACAO'], xbm_pred)



A matriz de confusão mostra as frequências de classificação para cada classe do modelo. Pegando o exemplo acima, ela vai nos mostrar as frequências:
Verdadeiro positivo (true positive — TP): ocorre quando no conjunto real, a classe que estamos buscando foi prevista corretamente. Por exemplo, quando a mulher está grávida e o modelo previu corretamente que ela está grávida.
Falso positivo (false positive — FP): ocorre quando no conjunto real, a classe que estamos buscando prever foi prevista incorretamente. Exemplo: a mulher não está grávida, mas o modelo disse que ela está.
Falso verdadeiro (true negative — TN): ocorre quando no conjunto real, a classe que não estamos buscando prever foi prevista corretamente. Exemplo: a mulher não estava grávida, e o modelo previu corretamente que ela não está.
Falso negativo (false negative — FN): ocorre quando no conjunto real, a classe que não estamos buscando prever foi prevista incorretamente. Por exemplo, quando a mulher está grávida e o modelo previu incorretamente que ela não está grávida.


Avaliando os gráficos representados pela matriz de confusão, podemos observar que ambos os modelos se comportaram
de forma bastante assertiva.



# Conclusão

Os dados iniciais possuíam algumas informações inconsistentes, o que exigiu um tratamento das informações para possibilitar
uma análise exploratória e viabilizar a aplicação dos modelos de predição.

Após o tratamento dos dados inconsistentes, foram utilizados 3 modelos de predição:  RandomForest, GBM e XBM, onde a acurácia foi de 91%, 96% e 94% respectivamente,
e para avaliar o desempenho dos modelos foi realizada a matriz de confusão, onde constatou-se graficamente que os modelos ficaram ajustados
